In [ ]:
# =========================
# GOES-IR (Colab)
# Descarga, realce clásico y KML de cuenca
# =========================

# 1) Instalación de dependencias (Colab)
!pip -q install s3fs fastkml shapely cartopy xarray netCDF4 h5netcdf pyproj gpxpy

In [66]:
# 2) Imports
import os, re, math, datetime as dt
import numpy as np
import numpy.ma as ma
import xarray as xr
import s3fs
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
from shapely.geometry import LineString, Polygon
from fastkml import kml
from pyproj import Proj, Transformer
from pathlib import Path
from scipy.interpolate import griddata

In [67]:
# 3) (Opcional) Montar Google Drive para leer KML y guardar PNG
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception:
    pass

Mounted at /content/drive


In [ ]:
# Indicar rango exacto:

N_VUELO = 69                        # Número de vuelo
START_UTC_STR = "2025-10-12 18:50"  # 30 min antes se la siembra
END_UTC_STR =   "2025-10-12 20:37"  # 30 min después se la siembra
FIG_START = 32                      # número de figura inicial (p.ej., 11)

In [ ]:
# 4) PARÁMETROS DEL LUGAR ================================
GOES_ID   = 19               # o 16 (East)
BAND      = 13               # Band 13 (IR "Clean LW")

LOCAL_OFFSET_HOURS = -5

# Track siembra
# debe tener 3 tracks con los siguientes nombres: inicio_v{N_VUELO}, fin_v{N_VUELO} y siembra_v{N_VUELO}
TRACK_GPX_PATH = f"/content/drive/MyDrive/2025/VUELO_{N_VUELO}.GPX"

# Bounding box aprox. Poligono en sector 1:
LAT_MIN, LAT_MAX = -3.5, -2.0
LON_MIN, LON_MAX = -79.5, -78.0
# Ruta al KML
#KML_PATH = "/content/drive/MyDrive/Poligono.kml" # sector 1

# Bounding box aprox. Poligono en sector 2:
#LAT_MIN, LAT_MAX = -0.670, 0.472
#LON_MIN, LON_MAX = -78.684, -77.728
# Ruta al KML
#KML_PATH = "/content/drive/MyDrive/PoligonoNorte.kml" # sector 2

# Bounding box aprox. Poligono en sector 3:
#LAT_MIN, LAT_MAX = -3.597, 0.505
#LON_MIN, LON_MAX = -79.705, -77.462
# Ruta al KML
KML_PATH = "/content/drive/MyDrive/PoligonoNorteCuenca.kml" 

# Carpeta de salida
OUT_DIR    = "/content/drive/MyDrive/2025"
os.makedirs(OUT_DIR, exist_ok=True)

In [70]:
# 5) Utilidades ==============================================
def parse_goes_start_time_from_key(key):
    """
    Extrae el start time del nombre de archivo GOES.
    Soporta fracción de segundo opcional después de SS (p. ej., '208' = 20.8 s).
    Devuelve datetime (UTC) ignorando la fracción.
    """
    import re, datetime as dt
    # _sYYYYJJJHHMMSS[fraction]_   (fraction opcional de 1-3 dígitos)
    m = re.search(r"_s(\d{4})(\d{3})(\d{2})(\d{2})(\d{2})(\d{0,3})_", str(key))
    if not m:
        return None
    year  = int(m.group(1))
    jday  = int(m.group(2))
    hh    = int(m.group(3))
    mm    = int(m.group(4))
    ss    = int(m.group(5))
    # fracción m.group(6) se ignora para el filtro (no la necesitas)
    start = dt.datetime(year, 1, 1) + dt.timedelta(days=jday-1, hours=hh, minutes=mm, seconds=ss)
    return start

def list_hour_keys(fs, bucket, product, year, jday, hour, band):
    pattern = f"s3://{bucket}/{product}/{year:04d}/{jday:03d}/{hour:02d}/*C{band:02d}_*.nc"
    try:
        fs.invalidate_cache(pattern)
    except Exception:
        pass
    try:
        keys = fs.glob(pattern)
    except Exception:
        keys = []
    return [str(k) for k in keys]

def iter_range_hours(start_dt, end_dt):
    # Genera (year, jday, hour) para cada hora entre start y end (inclusive)
    cur = start_dt.replace(minute=0, second=0, microsecond=0)
    last = end_dt.replace(minute=0, second=0, microsecond=0)
    while cur <= last:
        yield cur.year, int(cur.strftime("%j")), cur.hour
        cur += dt.timedelta(hours=1)

# --- Lector KML robusto: sanea tipos y tiene fallback con lxml ---
def read_kml_geometries(kml_path):
    """
    Devuelve una lista de geometrías Shapely (LineString/Polygon) en lon/lat.
    1) Sanea tipos no soportados por fastkml (long, ulong, int64, uint64).
    2) Intenta fastkml.
    3) Si falla, fallback con lxml leyendo <coordinates>.
    """
    import re
    from shapely.geometry import LineString, Polygon
    from fastkml import kml as fast_kml
    from fastkml.helpers import KMLParseError

    if not os.path.exists(kml_path):
        return []

    # 1) Leer y sanear tipos de SimpleField
    with open(kml_path, "rt", encoding="utf-8") as f:
        txt = f.read()

    # Reemplazos de tipos no soportados
    replacements = {
        r'type="long"':  'type="int"',
        r"type='long'":  "type='int'",
        r'type="ulong"': 'type="uint"',
        r"type='ulong'": "type='uint'",
        r'type="int64"': 'type="int"',
        r"type='int64'": "type='int'",
        r'type="uint64"':'type="uint"',
        r"type='uint64'":"type='uint'",
    }
    for pat, rep in replacements.items():
        txt = re.sub(pat, rep, txt, flags=re.IGNORECASE)

    # 2) Intentar fastkml
    try:
        kdoc = fast_kml.KML()
        kdoc.from_string(txt.encode("utf-8"))
        geoms = []

        def _collect(feat):
            if hasattr(feat, "features"):
                for sub in feat.features():
                    _collect(sub)
            if hasattr(feat, "geometry") and feat.geometry is not None:
                g = feat.geometry
                try:
                    # Puede ser LineString, Polygon, Multi*, GeometryCollection
                    if isinstance(g, (LineString, Polygon)):
                        geoms.append(g)
                    else:
                        for gg in getattr(g, "geoms", []):
                            if isinstance(gg, (LineString, Polygon)):
                                geoms.append(gg)
                except Exception:
                    pass

        for f0 in kdoc.features():
            _collect(f0)

        if geoms:
            return geoms
        # Si no encontró nada, cae al fallback
        raise ValueError("fastkml no encontró geometrías; uso fallback.")

    except (KMLParseError, ValueError, Exception) as e:
        print("Aviso: fastkml no pudo parsear; usando parser flexible con lxml. Detalle:", repr(e))
        # 3) Fallback con lxml: buscar coordinates en LineString/Polygon
        try:
            from lxml import etree
        except Exception:
            # Instalar lxml si faltara (raro en Colab)
            import sys
            !{sys.executable} -m pip -q install lxml
            from lxml import etree

        ns = {
            "kml": "http://www.opengis.net/kml/2.2",
            "gx":  "http://www.google.com/kml/ext/2.2",
        }
        tree = etree.parse(kml_path)
        root = tree.getroot()

        def parse_coords(text):
            pts = []
            for token in (text or "").strip().replace("\n", " ").split():
                parts = token.split(",")
                if len(parts) >= 2:
                    try:
                        lon = float(parts[0]); lat = float(parts[1])
                        pts.append((lon, lat))
                    except Exception:
                        pass
            return pts

        geoms = []

        # LineString
        for node in root.findall(".//kml:LineString/kml:coordinates", ns):
            pts = parse_coords(node.text)
            if len(pts) >= 2:
                geoms.append(LineString(pts))

        # Polygon (outerBoundary)
        for node in root.findall(".//kml:Polygon/kml:outerBoundaryIs/kml:LinearRing/kml:coordinates", ns):
            pts = parse_coords(node.text)
            if len(pts) >= 3:
                geoms.append(Polygon(pts))

        # MultiGeometry: ya queda cubierto porque buscamos por debajo con // (descendientes)

        return geoms

from matplotlib.colors import LinearSegmentedColormap

from matplotlib.colors import LinearSegmentedColormap

def build_geos_cartopy_crs(ds):
    proj = ds['goes_imager_projection']
    sat_h = float(proj.perspective_point_height)
    lon0  = float(proj.longitude_of_projection_origin)
    a     = float(proj.semi_major_axis)
    b     = float(proj.semi_minor_axis)
    globe = ccrs.Globe(semimajor_axis=a, semiminor_axis=b)
    geos = ccrs.Geostationary(central_longitude=lon0, satellite_height=sat_h, globe=globe, sweep_axis='x')
    return geos, sat_h, lon0, a, b

def lonlat_to_geos_xy_meters(lons, lats, sat_h, lon0, a, b):
    p_geos = Proj(proj='geos', h=sat_h, lon_0=lon0, a=a, b=b, sweep='x')
    transformer = Transformer.from_crs("EPSG:4326", p_geos.crs, always_xy=True)
    xm, ym = transformer.transform(lons, lats)
    return np.array(xm), np.array(ym)

def meters_to_radians(x_m, y_m, sat_h):
    return np.arctan(x_m / sat_h), np.arctan(y_m / sat_h)

In [71]:
# 6) Rango temporal ==========================================
if START_UTC_STR and END_UTC_STR:
    start_dt = dt.datetime.strptime(START_UTC_STR, "%Y-%m-%d %H:%M")
    end_dt   = dt.datetime.strptime(END_UTC_STR,   "%Y-%m-%d %H:%M")
else:
    base = dt.datetime.strptime(DATE_UTC, "%Y-%m-%d")
    start_dt = base.replace(hour=HOUR_START_UTC, minute=0, second=0, microsecond=0)
    end_dt   = base.replace(hour=HOUR_END_UTC,   minute=59, second=59, microsecond=0)

if end_dt < start_dt:
    raise ValueError("END_UTC es anterior a START_UTC. Ajusta el rango.")

print(f"Rango UTC: {start_dt}  →  {end_dt}")

Rango UTC: 2025-10-12 18:50:00  →  2025-10-12 20:37:00


In [72]:
# ============================================================
# BOOTSTRAP TRAS REINICIO (definir TODO lo necesario)
# ============================================================
import re, datetime as dt
import numpy as np
import s3fs, xarray as xr

# --- Construir rango temporal ---
if START_UTC_STR and END_UTC_STR:
    start_dt = dt.datetime.strptime(START_UTC_STR, "%Y-%m-%d %H:%M")
    end_dt   = dt.datetime.strptime(END_UTC_STR,   "%Y-%m-%d %H:%M")
else:
    base     = dt.datetime.strptime(DATE_UTC, "%Y-%m-%d")
    start_dt = base.replace(hour=HOUR_START_UTC, minute=0, second=0, microsecond=0)
    end_dt   = base.replace(hour=HOUR_END_UTC,   minute=59, second=59, microsecond=0)
if end_dt < start_dt:
    raise ValueError("END_UTC anterior a START_UTC")

# --- S3: bucket/product/fs ---
bucket  = f"noaa-goes{GOES_ID}"
product = "ABI-L2-CMIPF"    # Full Disk L2 CMIPF
fs = s3fs.S3FileSystem(anon=True)

# --- Utilidades robustas ---
def parse_goes_start_time_from_key(key):
    # Acepta fracción de segundo opcional en SSf
    m = re.search(r"_s(\d{4})(\d{3})(\d{2})(\d{2})(\d{2})(\d{0,3})_", str(key))
    if not m:
        return None
    year  = int(m.group(1)); jday = int(m.group(2))
    hh    = int(m.group(3)); mm   = int(m.group(4)); ss = int(m.group(5))
    return dt.datetime(year, 1, 1) + dt.timedelta(days=jday-1, hours=hh, minutes=mm, seconds=ss)

def list_hour_keys(fs, bucket, product, year, jday, hour, band):
    # Glob directo a la banda pedida (evita filtros frágiles)
    pattern = f"s3://{bucket}/{product}/{year:04d}/{jday:03d}/{hour:02d}/*C{band:02d}_*.nc"
    try:
        fs.invalidate_cache(pattern)
    except Exception:
        pass
    try:
        keys = fs.glob(pattern)
    except Exception:
        keys = []
    return [str(k) for k in keys]

def iter_range_hours(start_dt, end_dt):
    cur = start_dt.replace(minute=0, second=0, microsecond=0)
    last = end_dt.replace(minute=0, second=0, microsecond=0)
    while cur <= last:
        yield cur.year, int(cur.strftime("%j")), cur.hour
        cur += dt.timedelta(hours=1)

# --- Chequeo rápido del prefijo ---
print("Chequeo rápido del prefijo:")
test_prefix = f"s3://{bucket}/{product}/{start_dt.year:04d}/{int(start_dt.strftime('%j')):03d}/{start_dt.hour:02d}/"
print("Prefijo:", test_prefix)
try:
    sample = fs.ls(test_prefix)
    print("Objetos listados:", len(sample))
    if sample:
        print("Ejemplo:", sample[0])
except Exception as e:
    print("Falló ls():", repr(e))

# --- Buscar archivos en el rango ---
candidate_keys = []
for year, jday, hour in iter_range_hours(start_dt, end_dt):
    candidate_keys.extend(list_hour_keys(fs, bucket, product, year, jday, hour, BAND))

keys_in_range = []
for k in candidate_keys:
    t0 = parse_goes_start_time_from_key(k)
    if t0 and (start_dt <= t0 <= end_dt):
        keys_in_range.append((t0, k))
keys_in_range.sort()
print("Archivos en rango:", len(keys_in_range))

# Fallback ±1h si quedó vacío
if not keys_in_range:
    print("Sin escenas en ventana exacta; pruebo ±1h…")
    start_fbk = (start_dt - dt.timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    end_fbk   = (end_dt   + dt.timedelta(hours=1)).replace(minute=59, second=59, microsecond=0)
    candidate_keys = []
    for year, jday, hour in iter_range_hours(start_fbk, end_fbk):
        candidate_keys.extend(list_hour_keys(fs, bucket, product, year, jday, hour, BAND))
    keys_in_range = []
    for k in candidate_keys:
        t0 = parse_goes_start_time_from_key(k)
        if t0 and (start_fbk <= t0 <= end_fbk):
            keys_in_range.append((t0, k))
    keys_in_range.sort()
    print("Archivos tras fallback:", len(keys_in_range))
    if not keys_in_range:
        raise RuntimeError("No se encontraron escenas C{:02d} cerca del rango. Revisa GOES_ID/UTC o amplía ventana."
                           .format(BAND))

Chequeo rápido del prefijo:
Prefijo: s3://noaa-goes19/ABI-L2-CMIPF/2025/285/18/
Objetos listados: 96
Ejemplo: noaa-goes19/ABI-L2-CMIPF/2025/285/18/OR_ABI-L2-CMIPF-M6C01_G19_s20252851800212_e20252851809520_c20252851809583.nc
Archivos en rango: 11


In [73]:
print("Chequeo rápido del prefijo:")
test_prefix = f"s3://{bucket}/{product}/{start_dt.year:04d}/{int(start_dt.strftime('%j')):03d}/{start_dt.hour:02d}/"
print("Prefijo:", test_prefix)
try:
    sample = fs.ls(test_prefix)
    print("Objetos listados:", len(sample))
    if sample:
        print("Ejemplo:", sample[0])
except Exception as e:
    print("Falló el ls():", repr(e))

Chequeo rápido del prefijo:
Prefijo: s3://noaa-goes19/ABI-L2-CMIPF/2025/285/18/
Objetos listados: 96
Ejemplo: noaa-goes19/ABI-L2-CMIPF/2025/285/18/OR_ABI-L2-CMIPF-M6C01_G19_s20252851800212_e20252851809520_c20252851809583.nc


In [74]:
import re

test_prefix = f"s3://{bucket}/{product}/{start_dt.year:04d}/{int(start_dt.strftime('%j')):03d}/{start_dt.hour:02d}/"
objs = fs.ls(test_prefix)
bands = []
for k in objs:
    m = re.search(r"M\d+C(\d{2})_G\d{2}", str(k))
    if m:
        bands.append(int(m.group(1)))
bands_sorted = sorted(set(bands))
print("Bandas encontradas en la hora:", bands_sorted)

Bandas encontradas en la hora: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [75]:
def list_hour_keys(fs, bucket, product, year, jday, hour, band):
    """
    Devuelve las rutas s3://... de CMIPF para una hora dada, filtradas por banda (p.ej., C13).
    Usa glob para no depender de post-filtrado frágil por substrings.
    """
    # patrón directo a los netCDF de esa banda:
    pattern = f"s3://{bucket}/{product}/{year:04d}/{jday:03d}/{hour:02d}/*C{band:02d}_*.nc"
    try:
        fs.invalidate_cache(pattern)
    except Exception:
        pass
    try:
        keys = fs.glob(pattern)
    except FileNotFoundError:
        keys = []
    except Exception:
        keys = []
    # normaliza a str
    keys = [str(k) for k in keys]
    return keys

In [76]:
# 7) Buscar archivos en S3 (NOAA) ============================

print("Chequeo rápido del prefijo:")
test_prefix = f"s3://{bucket}/{product}/{start_dt.year:04d}/{int(start_dt.strftime('%j')):03d}/{start_dt.hour:02d}/"
print("Prefijo:", test_prefix)
try:
    sample = fs.ls(test_prefix)
    print("Objetos listados:", len(sample))
    if sample:
        print("Ejemplo:", sample[0])
except Exception as e:
    print("Falló el ls():", repr(e))



bucket  = f"noaa-goes{GOES_ID}"
product = "ABI-L2-CMIPF"   # Full Disk
fs = s3fs.S3FileSystem(anon=True)

candidate_keys = []
for year, jday, hour in iter_range_hours(start_dt, end_dt):
    try:
        keys = list_hour_keys(fs, bucket, product, year, jday, hour, BAND)
        candidate_keys.extend(keys)
    except Exception:
        pass

# filtrar por rango exacto de tiempo (inicio de escaneo <= t <= fin)
keys_in_range = []
for k in candidate_keys:
    t0 = parse_goes_start_time_from_key(k)
    if t0 and (start_dt <= t0 <= end_dt):
        keys_in_range.append((t0, k))

keys_in_range.sort()  # ascendente por tiempo
print(f"Archivos encontrados en rango: {len(keys_in_range)}")

if not keys_in_range:
    raise RuntimeError("No se encontraron escenas para el rango indicado (verifica UTC, banda o GOES_ID).")


Chequeo rápido del prefijo:
Prefijo: s3://noaa-goes19/ABI-L2-CMIPF/2025/285/18/
Objetos listados: 96
Ejemplo: noaa-goes19/ABI-L2-CMIPF/2025/285/18/OR_ABI-L2-CMIPF-M6C01_G19_s20252851800212_e20252851809520_c20252851809583.nc
Archivos encontrados en rango: 11


In [77]:
# 8) Leer KML
geoms = read_kml_geometries(KML_PATH)
if geoms:
    print(f"KML: {len(geoms)} geometrías leídas.")
else:
    print("Aviso: no se cargaron geometrías KML (ruta inválida o KML vacío).")


Aviso: fastkml no pudo parsear; usando parser flexible con lxml. Detalle: TypeError("'list' object is not callable")
KML: 1 geometrías leídas.


In [78]:
# 9) === Tracks desde GPX (BaseCamp) ============================================

import gpxpy, re, numpy as np

# Ahora incluye 'pausa'
NAME_REGEX = re.compile(r'(?P<tipo>inicio|siembra|fin|pausa)[ _-]*v(?P<flight>\d+)', re.I)

def load_gpx_grouped(path):
    # agregamos la clave 'pausa' en el dict base
    flights = {}  # {'v1': {'inicio':[], 'siembra':[], 'fin':[], 'pausa':[]}, ...}
    with open(path, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)
    for trk in gpx.tracks:
        name = (trk.name or "").lower()
        m = NAME_REGEX.search(name)
        if m:
            tipo = m.group("tipo")
            flight = f"v{int(m.group('flight'))}"
        else:
            # Fallback: buscar por palabra clave
            if "siembra" in name:
                tipo = "siembra"
            elif "inicio" in name:
                tipo = "inicio"
            elif "fin" in name:
                tipo = "fin"
            elif "pausa" in name:
                tipo = "pausa"
            else:
                continue
            flight = "misc"

        d = flights.setdefault(flight, {"inicio": [], "siembra": [], "fin": [], "pausa": []})
        for seg in trk.segments:
            lon = np.array([p.longitude for p in seg.points], dtype=float)
            lat = np.array([p.latitude  for p in seg.points], dtype=float)
            if lon.size >= 2:
                d[tipo].append((lon, lat))
    return flights

flights = load_gpx_grouped(TRACK_GPX_PATH)
print("Vuelos cargados y segmentos por tipo:")
print({k: {t: len(v) for t, v in d.items()} for k, d in flights.items()})

FLIGHT_FILTER = None

Vuelos cargados y segmentos por tipo:
{'v69': {'inicio': 0, 'siembra': 1, 'fin': 1, 'pausa': 0}, 'misc': {'inicio': 1, 'siembra': 0, 'fin': 0, 'pausa': 0}}


In [ ]:
# 10) Loop de ploteo =========================================
# Requiere: keys_in_range, fs, xr, build_geos_cartopy_crs,
# lonlat_to_geos_xy_meters, meters_to_radians, LON_MIN/MAX, LAT_MIN/MAX,
# GOES_ID, BAND, OUT_DIR, geoms ya definidos.

from pathlib import Path
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import griddata

# === Numeración de figuras ===
ADD_NUM_IN_FILENAME = True      # True: agrega "FigXX_" al nombre del archivo

# --- Paleta EXACTA (°C) como la referencia (tu versión) ---
def make_ir_palette_exact():
    C = lambda c: c + 273.15  # °C -> K
    VMIN = C(-110.0)          # 163.15 K
    VMAX = C(57.0)            # 330.15 K
    # Duplicamos -59 °C para salto negro→rojo
    stops = [
        (C(-110),  "#6b016b"),  # Fuxia
        (C(-100),  "#6b016b"),  # Fuxia
        (C(-81),   "#f5b3f5"),  # rosado
        (C(-80),   "#FFFFFF"),  # blanco
        (C(-68.10),"#000000"),  # negro justo antes del corte
        (C(-59.00),"#FF0000"),  # rojo (inicio realce)
        (C(-49.0), "#f7f719"),  # amarillo
        (C(-40.0), "#00FF00"),  # verde
        (C(-27.0), "#031d59"),  # azul oscuro
        (C(-17.0), "#00FFFF"),  # cian
        (C(-16.0), "#c7fdff"),  #
        (C(-15.0), "#FFFFFF"),  #
        #(C(-1.0),  "#FFFFFF"),  # Isoterma 0
        #(C(0.0),   "#0905f7"),  # Isoterma 0
        #(C(1.0),   "#FFFFFF"),  # Isoterma 0
        (C(6.0),   "#A0A0A0"),
        (C(31.0),  "#606060"),
        (C(57.0),  "#000000"),
    ]
    pos  = [(t-VMIN)/(VMAX-VMIN) for t,_ in stops]
    cols = [c for _, c in stops]
    cmap = LinearSegmentedColormap.from_list("IR_exact_C", list(zip(pos, cols)))
    cmap = cmap.copy()
    cmap.set_bad((1,1,1,0))   # NaN transparentes
    cmap.set_under("#FFFFFF")
    cmap.set_over("#000000")
    return cmap, VMIN, VMAX

ir_cmap, VMIN, VMAX = make_ir_palette_exact()

def safe_slice_1d(coord, v0, v1):
    c0, c1 = float(coord[0]), float(coord[-1])
    cmin, cmax = (c0, c1) if c0 <= c1 else (c1, c0)
    lo, hi = (v0, v1) if v0 <= v1 else (v1, v0)
    lo = max(min(lo, cmax), cmin)
    hi = max(min(hi, cmax), cmin)
    return slice(lo, hi) if c0 <= c1 else slice(hi, lo)

# Resolución del remuestreo (°). 0.02 ≈ ~2 km cerca de 30°S
RES_DEG = 0.02

for idx, (t0, key) in enumerate(keys_in_range):
    fig_num = FIG_START + idx  # numeración secuencial
    print(f"Procesando {t0} -> {key.split('/')[-1]} (Figura {fig_num})")

    # Abrir escena
    ds = xr.open_dataset(fs.open(key), engine="h5netcdf")
    geos_crs, sat_h, lon0, a, b = build_geos_cartopy_crs(ds)
    bt = ds["CMI"]; x = ds["x"]; y = ds["y"]

    # --- Recorte anticipado usando bbox lon/lat -> GEOS (m) -> rad ---
    try:
        x_m, y_m = lonlat_to_geos_xy_meters(
            np.array([LON_MIN, LON_MAX, LON_MIN, LON_MAX]),
            np.array([LAT_MIN, LAT_MIN, LAT_MAX, LAT_MAX]),
            sat_h, lon0, a, b
        )
        xmin_r, xmax_r = meters_to_radians(np.min(x_m), np.max(x_m), sat_h)
        ymin_r, ymax_r = meters_to_radians(np.min(y_m), np.max(y_m), sat_h)
        print(f"x req rad: {xmin_r:.5f}..{xmax_r:.5f} | x data: {float(x.min()):.5f}..{float(x.max()):.5f}")
        print(f"y req rad: {ymin_r:.5f}..{ymax_r:.5f} | y data: {float(y.min()):.5f}..{float(y.max()):.5f}")
        ds_sub = ds.sel(x=safe_slice_1d(x, xmin_r, xmax_r),
                        y=safe_slice_1d(y, ymin_r, ymax_r))
        bt_sub = ds_sub["CMI"]; xs = ds_sub["x"].values; ys = ds_sub["y"].values
    except Exception as e:
        print("Fallo en recorte; uso full-disk. Detalle:", repr(e))
        bt_sub, xs, ys = bt, x.values, y.values

    # Datos
    data = np.asarray(bt_sub.values, dtype=np.float32)
    if not np.isfinite(data).any():
        print("Subset sin datos finitos; uso full-disk.")
        bt_sub, xs, ys = bt, x.values, y.values
        data = np.asarray(bt_sub.values, dtype=np.float32)

    # --- GEOS -> lon/lat (convertir SOLO donde hay datos finitos) ---
    lon_g = np.arange(LON_MIN, LON_MAX + RES_DEG, RES_DEG)
    lat_g = np.arange(LAT_MIN, LAT_MAX + RES_DEG, RES_DEG)
    LONg, LATg = np.meshgrid(lon_g, lat_g)

    Xr, Yr = np.meshgrid(xs, ys)                 # (Ny,Nx)
    Xm = sat_h * np.tan(Xr); Ym = sat_h * np.tan(Yr)

    from pyproj import Proj, Transformer
    p_geos = Proj(proj="geos", h=sat_h, lon_0=lon0, a=a, b=b, sweep="x")
    t_inv = Transformer.from_crs(p_geos.crs, "EPSG:4326", always_xy=True)
    lon2d, lat2d = t_inv.transform(Xm, Ym)

    good = np.isfinite(lon2d) & np.isfinite(lat2d) & np.isfinite(data)
    n_good = int(good.sum())
    print(f"Pixeles válidos para remuestrear: {n_good}")
    fallback_geos = (n_good == 0)

    # --- Hora UTC y local ---
    try:
        from zoneinfo import ZoneInfo
        t_utc = t0.replace(tzinfo=dt.timezone.utc)        # t0 es UTC naive en tu código
        t_loc = t_utc.astimezone(ZoneInfo(LOCAL_TZ))
        off_h = int(t_loc.utcoffset().total_seconds() // 3600)
    except Exception:
        # Fallback simple si no está zoneinfo
        t_utc = t0
        t_loc = t0 + dt.timedelta(hours=LOCAL_OFFSET_HOURS)
        off_h = LOCAL_OFFSET_HOURS

    utc_str = t_utc.strftime('%Y-%m-%d %H:%M') + " UTC"
    loc_str = f"{t_loc.strftime('%Y-%m-%d %H:%M')} local (UTC{off_h:+d})"



    fig = plt.figure(figsize=(9.5, 9))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_facecolor("white")
    ax.set_title(
        f"GOES-{GOES_ID} Band {BAND} (IR) • Vuelo N° {N_VUELO}\n{utc_str} / {loc_str}",
        fontsize=12, weight="bold"
    )

    if not fallback_geos:
        pts  = np.column_stack([lon2d[good].ravel(), lat2d[good].ravel()])
        vals = data[good].ravel()
        grid = griddata(pts, vals, (LONg, LATg), method="nearest")

        mappable = ax.imshow(
            grid, origin="lower",
            extent=(lon_g.min(), lon_g.max(), lat_g.min(), lat_g.max()),
            transform=ccrs.PlateCarree(),
            cmap=ir_cmap, vmin=VMIN, vmax=VMAX,
            interpolation="nearest", zorder=0
        )
    else:
        img_ma = ma.masked_invalid(data)
        mappable = ax.imshow(
            img_ma, origin="upper",
            extent=(float(xs.min()), float(xs.max()), float(ys.min()), float(ys.max())),
            transform=geos_crs,
            cmap=ir_cmap, vmin=VMIN, vmax=VMAX,
            interpolation="nearest", zorder=0
        )

    # Recorte visual al bbox
    ax.set_extent([LON_MIN, LON_MAX, LAT_MIN, LAT_MAX], crs=ccrs.PlateCarree())

    # Costas/bordes + KML
    ax.coastlines("10m", linewidth=0.8, color="#3b3a3b", zorder=3)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor="#3b3a3b", zorder=3)
    if 'geoms' in globals() and geoms:
        ax.add_geometries(geoms, crs=ccrs.PlateCarree(), facecolor="none",
                          edgecolor="black", linewidth=1.8, zorder=4)

    # --- Puntos de referencia ---
    ref_points = {
        "Otavalo": (0.236, -78.272),
        "Cayambe": (0.042, -78.148),
    }

    for name, (latp, lonp) in ref_points.items():
        # Verificar si el punto está dentro del área definida
        if (LAT_MIN <= latp <= LAT_MAX) and (LON_MIN <= lonp <= LON_MAX):
            # marcador
            ax.plot(
                lonp, latp,
                marker="o", markersize=5,
                markeredgecolor="black", markerfacecolor="yellow",
                transform=ccrs.PlateCarree(), zorder=7
            )
            # etiqueta levemente desplazada a la derecha
            ax.text(
                lonp + 0.02, latp, name,
                transform=ccrs.PlateCarree(),
                fontsize=8, color="black",
                bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=1.5),
                zorder=7
            )
        else:
            print(f"{name} está fuera del área ({latp:.3f}, {lonp:.3f}), no se grafica.")

    # Grid y colorbar (°C) con ticks fijos
    gl = ax.gridlines(draw_labels=True, linewidth=0.3, color="k", alpha=0.2, linestyle="--")
    gl.top_labels = False; gl.right_labels = False
    ax.xaxis.set_major_formatter(LongitudeFormatter()); ax.yaxis.set_major_formatter(LatitudeFormatter())

    ticks_c = [-110, -59, -20, 6, 31, 57]
    ticks_k = [t + 273.15 for t in ticks_c]
    cb = plt.colorbar(mappable, ax=ax, orientation="vertical", shrink=0.8, pad=0.02)
    cb.set_label("Temperatura de brillo (°C)", fontsize=10)
    cb.set_ticks(ticks_k); cb.set_ticklabels([str(t) for t in ticks_c])

    # === Dibujar tracks y leyenda (si cargaste 'flights') ======================
    COLORS = {
        "inicio": "#7f7f7f",   # gris
        "siembra": "#ff00ff",  # magenta
        "fin": "#7f7f7f",      # gris
        "pausa": "#7f7f7f",    # gris
    }

    if 'flights' in globals() and flights:
        for flight_id, flight_data in flights.items():
            # si hay filtro, respétalo
            if 'FLIGHT_FILTER' in globals() and FLIGHT_FILTER and flight_id not in FLIGHT_FILTER:
                continue

            # ahora recorremos también 'pausa'
            for tipo in ("inicio", "siembra", "fin", "pausa"):
                # puede que ese vuelo no tenga ese tipo
                if tipo not in flight_data:
                    continue
                for lon, lat in flight_data[tipo]:
                    ax.plot(
                        lon, lat,
                        transform=ccrs.PlateCarree(),
                        color=COLORS[tipo],
                        linewidth=2.2,
                        zorder=6
                    )

        from matplotlib.lines import Line2D
        handles = [
            Line2D([0], [0], color=COLORS["inicio"],  lw=2.5, label="Ida"),
            Line2D([0], [0], color=COLORS["siembra"], lw=2.5, label="Siembra"),
            Line2D([0], [0], color=COLORS["fin"],     lw=2.5, label="Regreso"),
            Line2D([0], [0], color=COLORS["pausa"],   lw=2.5, label="Pausa"),
        ]
        ax.legend(
            handles=handles,
            loc="lower right",
            frameon=True,
            facecolor="white",
            framealpha=0.85
        )

    # === Número de figura en la imagen ========================================
    ax.text(
        0.01, 0.99, f"Figura {fig_num}",
        transform=ax.transAxes, ha="left", va="top",
        fontsize=12, weight="bold", color="black",
        bbox=dict(facecolor="white", alpha=0.85, edgecolor="none", boxstyle="round,pad=0.25"),
        zorder=10
    )

    # Guardar
    base_name = f"GOES{GOES_ID}_B{BAND:02d}_{t0.strftime('%Y%m%d_%H%M')}"
    if ADD_NUM_IN_FILENAME:
        out_name = f"Fig{fig_num:02d}_{base_name}.png"
    else:
        out_name = f"{base_name}.png"

    plt.savefig(str(Path(OUT_DIR) / out_name), dpi=220, bbox_inches="tight")
    plt.close(fig)

print(f"\nListo. PNGs guardados en: {OUT_DIR}")

Procesando 2025-10-12 18:50:21 -> OR_ABI-L2-CMIPF-M6C13_G19_s20252851850212_e20252851859533_c20252851859583.nc (Figura 32)
x req rad: -0.01396..-0.00930 | x data: -0.15184..0.15184
y req rad: -0.01080..-0.00617 | y data: -0.15184..0.15184
Pixeles válidos para remuestrear: 6889
Otavalo está fuera del área (0.236, -78.272), no se grafica.
Cayambe está fuera del área (0.042, -78.148), no se grafica.
Procesando 2025-10-12 19:00:21 -> OR_ABI-L2-CMIPF-M6C13_G19_s20252851900212_e20252851909532_c20252851910004.nc (Figura 33)
x req rad: -0.01396..-0.00930 | x data: -0.15184..0.15184
y req rad: -0.01080..-0.00617 | y data: -0.15184..0.15184
Pixeles válidos para remuestrear: 6889
Otavalo está fuera del área (0.236, -78.272), no se grafica.
Cayambe está fuera del área (0.042, -78.148), no se grafica.
Procesando 2025-10-12 19:10:21 -> OR_ABI-L2-CMIPF-M6C13_G19_s20252851910212_e20252851919532_c20252851919590.nc (Figura 34)
x req rad: -0.01396..-0.00930 | x data: -0.15184..0.15184
y req rad: -0.01080

In [80]:
print(VMIN)
print(VMAX)

163.14999999999998
330.15
